In [2]:
"""
Text-based parser for ProteinNet Records.
"""

__author__ = "Mohammed AlQuraishi"
__copyright__ = "Copyright 2019, Harvard Medical School"
__license__ = "MIT"

#!/usr/bin/python

# imports
import sys
import re
import numpy as np

# Constants
NUM_DIMENSIONS = 3

# Functions for conversion from Mathematica protein files to TFRecords
_aa_dict = {'A': '0', 'C': '1', 'D': '2', 'E': '3', 'F': '4', 'G': '5', 'H': '6', 'I': '7', 'K': '8', 'L': '9', 'M': '10', 'N': '11', 'P': '12', 'Q': '13', 'R': '14', 'S': '15', 'T': '16', 'V': '17', 'W': '18', 'Y': '19'}
_dssp_dict = {'L': '0', 'H': '1', 'B': '2', 'E': '3', 'G': '4', 'I': '5', 'T': '6', 'S': '7'}
_mask_dict = {'-': '0', '+': '1'}


def letter_to_num(string, dict_):
    """ Convert string of letters to list of ints """
    patt = re.compile('[' + ''.join(dict_.keys()) + ']')
    num_string = patt.sub(lambda m: dict_[m.group(0)] + ' ', string)
    num = [int(i) for i in num_string.split()]
    return num

def read_record(file_, num_evo_entries):
    """ Read a Mathematica protein record from file and convert into dict. """
    
    dict_ = {}
    
            
    while True:
        next_line = file_.readline()
        if next_line == '[ID]' + '\n':
            id_ = file_.readline()[:-1]
            dict_.update({'id': id_})
        elif next_line == '[PRIMARY]' + '\n':
            primary = letter_to_num(file_.readline()[:-1], _aa_dict)
            dict_.update({'primary': primary})
        elif next_line == '[EVOLUTIONARY]' + '\n':
            evolutionary = []
            for residue in range(num_evo_entries): evolutionary.append([float(step) for step in file_.readline().split()])
            dict_.update({'evolutionary': evolutionary})
        elif next_line == '[SECONDARY]' + '\n':
            print(next_line)
            secondary = letter_to_num(file_.readline()[:-1], _dssp_dict)
            dict_.update({'secondary': secondary})
        elif next_line == '[TERTIARY]' + '\n':
            tertiary = []
            for axis in range(NUM_DIMENSIONS): tertiary.append([float(coord) for coord in file_.readline().split()])
            dict_.update({'tertiary': tertiary})
        elif next_line == '[MASK]' + '\n':
            mask = letter_to_num(file_.readline()[:-1], _mask_dict)
            dict_.update({'mask': mask})
        elif next_line == '\n':
            return dict_
        elif next_line == '':
            return None
        
        
    


In [2]:
def dict_to_protein_shape(dict_):
    protein = []
    amino_acid_f = []
    for i in range(len(dict_['primary'])):
        #primary structure
        prim_lab = [0] * 21
        index = dict_['primary'][i]
        prim_lab[index] = 1
    
        #secondary label
        sec_lab = [0] * 8
    
        #PSSM
        pssm = []
        for j in range(21):
            pssm.append(dict_['evolutionary'][j][i])
        
        amino_acid_f = prim_lab + sec_lab + pssm    
        protein.append(amino_acid_f)
        amino_acid_f = []

    while (len (protein) < 700):
        no_seq = [0] * 50
        protein.append(no_seq)
    
    #returns protein in shape (1,700,50)
    protein = np.array(protein)   
    return protein    

In [3]:
import gzip
def save_parti_dataset(proteins, caspName , index):
    
    dataset = np.zeros(( len(proteins), 700, 50))

    for i in range ( len(proteins) ):
        dataset[i] = proteins[i]
    
    f = gzip.GzipFile( caspName + "-" + str(index) + '.npy.gz', "w")
    np.save(f, dataset)
    f.close()
    print(dataset.shape)

In [ ]:
# main. accepts two command-line arguments: input file and the number of entries in evo profiles, and outputs dicts to stdout

input_path =   "casp12/training_30"  #= sys.argv[1] 
num_evo_entries = 21 #int(sys.argv[2]) if len(sys.argv) == 3 else 20 # default number of evo entries

input_file = open(input_path, 'r')
   
proteins = [] #np.zeros((1,700,50)) 

i = 0
datIndex = 0
transkaya_conter = 0
while True:
    
    #progress bar
    if i % 1000 == 0:
        print(i)
        
    #save first partition of dataset - same size as transkaya        
    if transkaya_conter == 5278:
        print("saving dataset")
        save_parti_dataset(proteins, input_path, datIndex)
        proteins = []
        transkaya_conter = 0
        datIndex += 1
    
    #reading file
    dict_ = read_record(input_file, num_evo_entries)
    if dict_ is not None:
        protein = dict_to_protein_shape(dict_)
        
        #700 is max len for proteins
        if (protein.shape[0] == 700):
            proteins.append(protein)
            transkaya_conter += 1
            i += 1 
            #proteins = np.vstack((proteins,protein))
    else:
        input_file.close()
        break
    

#removing the first elemnt used for stacking    
#proteins = np.delete(proteins, 0)    

In [ ]:
import mmap

#secondary label is not present in the files
with open('casp12/training_100', 'rb', 0) as file, \
     mmap.mmap(file.fileno(), 0, access=mmap.ACCESS_READ) as s:
    if s.find(b'SECONDARY') != -1:
        print('true')
    else: 
        print('false')

In [5]:
f = gzip.GzipFile( 'casp12/training_30-' + str(0) +'.npy.gz', "r")
dataset_part =  np.load(f)

In [28]:
#finding the number of total proteins before creating numpy on disk
nr_windows = 0
for i in range(4):
    print(i)
    f = gzip.GzipFile( 'casp12/training_30-' + str(i) + 'window19Middle-repeating-left' +'.npy.gz', "r")
    dataset_part =  np.load(f)
    print(dataset_part.shape[0], " shape of dataset")
    nr_windows += dataset_part.shape[0]
    print("all togheter ", nr_windows)
    del dataset_part

0
1094617  shape of dataset
all togheter  1094617
1
1074768  shape of dataset
all togheter  2169385
2
1090451  shape of dataset
all togheter  3259836
3
1095262  shape of dataset
all togheter  4355098


In [ ]:
filename = "casp12/proteinNet-all-windows-19-middle-repeating-left"
fp = np.memmap(filename, dtype='float64', mode='w+', shape=(4355098, 19, 50))

In [6]:
#adding all the information to big dataset
l = 0
r = 0
for i in range(4):
    
    f = gzip.GzipFile( 'casp12/training_30-' + str(i) + 'window19Middle-repeating-left' +'.npy.gz', "r")
    dataset_part =  np.load(f)
    print(dataset_part.shape)
    
    r += dataset_part.shape[0]
    print(l, " ", r)
    fp[l:r] =  dataset_part
    l = r
    del dataset_part

(1094617, 19, 50)
0   1094617
(1074768, 19, 50)
1094617   2169385
(1090451, 19, 50)
2169385   3259836
(1095262, 19, 50)
3259836   4355098


In [7]:
fp.flush()
del fp

In [ ]:
g = gzip.GzipFile( filename +'.npy.gz', "w")
np.save(g, fp)
g.close()

In [4]:
import gzip
import numpy as np
f = gzip.GzipFile( filename +'.npy.gz', "r")
dataset =  np.load(f)

print(dataset.shape)
print(dataset[0,0])

(4355098, 19, 50)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 1.]


In [8]:
filename = "casp12/proteinNet-30thinning-all-windows-19-middle-repeating-left"
fp = np.memmap(filename, dtype='float64', mode='r', shape=(4355098, 19, 50))

In [10]:
fp[0,0,]

memmap([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 1.])

In [ ]:
##creating nmap dataset with transkaya included

import gzip
f = gzip.GzipFile('datasetTranskay/X_train_window19Middle-repeating-left-right0.npy.gz', "r")
X_train_window = np.load(f)
print(X_train_window.shape)
print(X_train_window[0,0,22:30])

ada = np.zeros((X_train_window.shape[0], 19, 50))
ada[:, :, 0:21] = X_train_window[:,:,0:21]
ada[:, :, 21:29] = X_train_window[:,:,22:30]
ada[:, :, 29:] = X_train_window[:,:,35:56]
del X_train_window

print(ada.shape)
print(ada[0,0,21:29])

(1129862, 19, 57)
[1. 0. 0. 0. 0. 0. 0. 0.]


In [21]:
filename = "casp12/proteinNet-30thinning-all-windows-19-middle-repeating-left"
fp = np.memmap(filename, dtype='float64', mode='r', shape=(4355098, 19, 50))

filename = "casp12/proteinNet-30thinning-all-windows-19-middle-repeating-left-withTranskaya"
allFp = np.memmap(filename, dtype='float64', mode='w+', shape=(4355098 + ada.shape[0], 19, 50))

In [22]:
allFp[0:ada.shape[0]] = ada
allFp[ada.shape[0]:] = fp

allFp.flush()
del allFp

In [32]:
filename = "casp12/proteinNet-30thinning-all-windows-19-middle-repeating-left-withTranskaya"
allFp = np.memmap(filename, dtype='float64', mode='r', shape=(4355098 + ada.shape[0], 19, 50))

In [33]:
g = gzip.GzipFile( filename +'.npy.gz', "w")
np.save(g, allFp)
g.close()

In [3]:
#chechinkg if it the first part is indeed transkaya dataset
filename = "casp12/proteinNet-30thinning-all-windows-19-middle-repeating-left-withTranskaya"
fp = np.memmap(filename, dtype='float64', mode='r+', shape=(4355098 + 1129862, 19, 50))

In [ ]:
print(fp[1129862,0,21:29])
print(ada[1129861,0,21:29])

In [4]:
import pickle
from tensorflow.keras.models import load_model
indices = pickle.load( open( "allDataset_assembly/supervised-assembly-indices", "rb" ) )

assem_models = []
for i in range(len(indices)):
    path = 'allDataset_assembly/model-assmebly-' + str(indices[i]+1) + '.hdf5'
    m = load_model( path )
    assem_models.append(m)

In [5]:
assem_pred = []
for k in range(len(assem_models)):
    pred = assem_models[k].predict(fp[1129862:4355098+1129862,:,29:], verbose = 1)
    assem_pred.append(pred)    

with open('casp12/stacked-pred-on-proteinNet', 'wb') as file_pi:
        pickle.dump(assem_pred, file_pi)     

 912128/4355098 [=====>........................] - ETA: 39:00

KeyboardInterrupt: 